<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Vowpal Wabbit Deep Dive

<center>
<img src="https://github.com/VowpalWabbit/vowpal_wabbit/blob/master/logo_assets/vowpal-wabbits-github-logo.png?raw=true" height="30%" width="30%" alt="Vowpal Wabbit">
</center>

[Vowpal Wabbit](https://github.com/VowpalWabbit/vowpal_wabbit) is a fast online machine learning library that implements several algorithms relevant to the recommendation use case.

The main advantage of Vowpal Wabbit (VW) is that training is done in an online fashion typically using Stochastic Gradient Descent or similar variants, which allows it to scale well to very large datasets. Additionally, it is optimized to run very quickly and can support distributed training scenarios for extremely large datasets. 

VW is best applied to problems where the dataset is too large to fit into memory but can be stored on disk in a single node. Though distributed training is possible with additional setup and configuration of the nodes. The kinds of problems that VW handles well mostly fall into the supervised classification domain of machine learning (Linear Regression, Logistic Regression, Multiclass Classification, Support Vector Machines, Simple Neural Nets). It also supports Matrix Factorization approaches and Latent Dirichlet Allocation, as well as a few other algorithms (see the [wiki](https://github.com/VowpalWabbit/vowpal_wabbit/wiki) for more information).

A good example of a typical deployment use case is a Real Time Bidding scenario, where an auction to place an ad for a user is being decided in a matter of milliseconds. Feature information about the user and items must be extracted and passed into a model to predict likelihood of click (or other interaction) in short order. And if the user and context features are constantly changing (e.g. user browser and local time of day) it may be infeasible to score every possible input combination before hand. This is where VW provides value, as a platform to explore various algorithms offline to train a highly accurate model on a large set of historical data then deploy the model into production so it can generate rapid predictions in real time. Of course this isn't the only manner VW can be deployed, it is also possible to use it entirely online where the model is constantly updating, or use active learning approaches, or work completely offline in a pre-scoring mode. 

<h3>Vowpal Wabbit for Recommendations</h3>

In this notebook we demonstrate how to use the VW library to generate recommendations on the [Movielens](https://grouplens.org/datasets/movielens/) dataset.

Several things are worth noting in how VW is being used in this notebook:

By leveraging an Azure Data Science Virtual Machine ([DSVM](https://azure.microsoft.com/en-us/services/virtual-machines/data-science-virtual-machines/)), VW comes pre-installed and can be used directly from the command line. There are also python bindings to be able to use VW within a python environment and even a wrapper conforming to the SciKit-Learn Estimator API. However, the python bindings must be installed as an additional python package with Boost dependencies, so for simplicity's sake execution of VW is done via a subprocess call mimicking what would happen from the command line execution of the model.

VW expects a specific [input format](https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format), in this notebook to_vw() is a convenience function that converts the standard movielens dataset into the required data format. Datafiles are then written to disk and passed to VW for training.

The examples shown are to demonstrate functional capabilities of VW not to indicate performance advantages of different approaches. There are several hyper-parameters (e.g. learning rate and regularization terms) that can greatly impact performance of VW models which can be adjusted using [command line options](https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Command-Line-Arguments). To properly compare approaches it is helpful to learn about and tune these parameters on the relevant dataset.

<h3>Environment Setup</h3>

In [2]:
import os
import sys
from subprocess import run
from tempfile import TemporaryDirectory
from time import process_time

import pandas as pd
import papermill as pm

from reco_utils.common.notebook_utils import is_jupyter
from reco_utils.dataset.movielens import load_pandas_df
from reco_utils.dataset.python_splitters import python_stratified_split
from reco_utils.evaluation.python_evaluation import rmse, mae, exp_var, rsquared, get_top_k_items

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]
Pandas version: 0.23.4


In [3]:
def to_vw(df, output, logistic=False):
    """Convert Pandas DataFrame to vw input format
    Args:
        df (pd.DataFrame): input DataFrame
        output (str): path to output file
        logistic (bool): flag to convert label to logistic value
    """
    with open(output, 'w') as f:
        tmp = df.reset_index()

        # we need to reset the rating type to an integer to simplify the vw formatting
        tmp['rating'] = tmp['rating'].astype('int64')
        
        # convert rating to binary value
        if logistic:
            tmp['rating'] = tmp['rating'].apply(lambda x: 1 if x >= 3 else -1)
        
        # convert each row to VW input format (https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format)
        # [label] [tag]|[user namespace] [user id feature] |[item namespace] [movie id feature]
        # label is the true rating, tag is a unique id for the example just used to link predictions to truth
        # user and item namespaces separate the features to support interaction features through command line options
        for _, row in tmp.iterrows():
            f.write('{rating} {index}|user {userID} |item {itemID}\n'.format_map(row))

In [4]:
def run_vw(train_params, test_params, test_data, prediction_path, logistic=False):
    """Convenience function to train, test, and show metrics of interest
    Args:
        train_params (str): vw training parameters
        test_params (str): vw testing parameters
        test_data (pd.dataFrame): test data
        prediction_path (str): path to vw prediction output
        logistic (bool): flag to convert label to logistic value
    Returns:
        (dict): metrics and timing information
    """

    # train model
    train_start = process_time()
    run(train_params.split(' '), check=True)
    train_stop = process_time()
    
    # test model
    test_start = process_time()
    run(test_params.split(' '), check=True)
    test_stop = process_time()
    
    # read in predictions
    pred_df = pd.read_csv(prediction_path, delim_whitespace=True, names=['prediction'], index_col=1).join(test_data)
    
    test_df = test_data.copy()
    if logistic:
        # make the true label binary so that the metrics are captured correctly
        test_df['rating'] = test['rating'].apply(lambda x: 1 if x >= 3 else -1)
    else:
        # ensure results are integers in correct range
        pred_df['prediction'] = pred_df['prediction'].apply(lambda x: int(max(1, min(5, round(x)))))

    # calculate metrics
    result = dict()
    result['RMSE'] = rmse(test_df, pred_df)
    result['MAE'] = mae(test_df, pred_df)
    result['R2'] = rsquared(test_df, pred_df)
    result['Explained Variance'] = exp_var(test_df, pred_df)
    result['Train Time (ms)'] = (train_stop - train_start) * 1000
    result['Test Time (ms)'] = (test_stop - test_start) * 1000
    
    return result

In [19]:
# create temp directory to maintain data files
tmpdir = TemporaryDirectory()

model_path = os.path.join(tmpdir.name, 'vw.model')
train_path = os.path.join(tmpdir.name, 'train.dat')
test_path = os.path.join(tmpdir.name, 'test.dat')
train_logistic_path = os.path.join(tmpdir.name, 'train_logistic.dat')
test_logistic_path = os.path.join(tmpdir.name, 'test_logistic.dat')
prediction_path = os.path.join(tmpdir.name, 'prediction.dat')

<h3>Load & Transform Data</h3>

In [ ]:
# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

In [20]:
# load movielens data (use the 1M dataset)
df = load_pandas_df(MOVIELENS_DATA_SIZE)

# split data to train and test sets, default values take 75% of each users ratings as train, and 25% as test
train, test = python_stratified_split(df)

# save train and test data in vw format
to_vw(df=train, output=train_path)
to_vw(df=test, output=test_path)

# save data for logistic regression (requires adjusting the label)
to_vw(df=train, output=train_logistic_path, logistic=True)
to_vw(df=test, output=test_logistic_path, logistic=True)

<h3>Regression Based Recommendations</h3>

When considering different approaches for solving a problem with machine learning it is helpful to generate a baseline approach to understand how more complex solutions perform across dimensions of performance, time, and resource (memory or cpu) usage.

Regression based approaches are some of the simplest and fastest baselines to consider for many ML problems.

<h4> Linear Regression</h4>

As the data provides a numerical rating between 1-5, fitting those values with a linear regression model is easy approach. This model is trained on examples of ratings as the target variable and corresponding user ids and movie ids as independent features.

By passing each user-item rating in as an example the model will begin to learn weights based on average ratings for each user as well as average ratings per item.

This however can generate predicted ratings which are no longer integers, so some additional adjustments should be made at prediction time to convert them back to the integer scale of 1 through 5 if necessary. Here, this is done in the evaluate function.

In [23]:
"""
Quick description of command line parameters used
  Other optional parameters can be found here: https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Command-Line-Arguments
  VW uses linear regression by default, so no extra command line options
  -f <model_path>: indicates where the final model file will reside after training
  -d <data_path>: indicates which data file to use for training or testing
  -i <model_path>: indicates where to load the previously model file created during training
  -t: this executes inference only (no learned updates to the model)
  -p <prediction_path>: indicates where to store prediction output
"""
train_params = 'vw -f {model} -d {data}'.format(model=model_path, data=train_path)
test_params = 'vw -i {model} -d {data} -t -p {pred}'.format(model=model_path, data=test_path, pred=prediction_path)

result = run_vw(train_params=train_params, 
                test_params=test_params, 
                test_data=test, 
                prediction_path=prediction_path)

comparison = pd.DataFrame(result, index=['Linear Regression'])
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.960662,0.686874,0.261995,0.262026,7.232512,4.612065


<h4>Linear Regression with Interaction Features</h4>

Previously we treated the user features and item features independently, but taking into account interactions between features can provide a mechanism to learn more fine grained preferences of the users.

To generate interaction features use the quadratic command line argument and specify the namespaces that should be combined: '-q ui' combines the user and item namespaces based on the first letter of each.

Currently the userIDs and itemIDs used are integers which means the feature ID is used directly, for instance when user ID 123 rates movie 456, the training example puts a 1 in the values for features 123 and 456. However when interaction is specified (or if a feature is a string) the resulting interaction feature is hashed into the available feature space. Feature hashing is a way to take a very sparse high dimensional feature space and reduce it into a lower dimensional space. This allows for reduced memory while retaining fast computation of feature and model weights.

The caveat with feature hashing, is that it can lead to hash collisions, where separate features are mapped to the same location. In this case it can be beneficial to increase the size of the space to support interactions between features of high cardinality. The available feature space is dictated by the --bit_precision (-b) <N> argument. Where the total available space for all features in the model is 2<sup>N</sup>. 

See [Feature Hashing and Extraction](https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Feature-Hashing-and-Extraction) for more details.

In [24]:
"""
Quick description of command line parameters used
  -b <N>: sets the memory size to 2<sup>N</sup> entries
  -q <ab>: create quadratic feature interactions between features in namespaces starting with 'a' and 'b' 
"""
train_params = 'vw -b 26 -q ui -f {model} -d {data}'.format(model=model_path, data=train_path)
test_params = 'vw -i {model} -d {data} -t -p {pred}'.format(model=model_path, data=test_path, pred=prediction_path)

result = run_vw(train_params=train_params,
                test_params=test_params,
                test_data=test,
                prediction_path=prediction_path)

comparison = comparison.append(pd.DataFrame(result, index=['Linear Regression w/ Interaction']))
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.960662,0.686874,0.261995,0.262026,7.232512,4.612065
Linear Regression w/ Interaction,0.959608,0.688242,0.263613,0.263833,7.564942,4.993011


<h4> Multinomial Logistic Regression</h4>

An alternative to linear regression is to leverage multinomial logistic regression, or multiclass classification, which treats each rating value as a distinct class. 

This avoids any non integer results, but also reduces the training data for each class which could lead to poorer performance if the counts of different rating levels are skewed.

Basic multiclass logistic regression can be accomplished using the One Against All approach specified by the '--oaa N' option, where N is the number of classes and proving the logistic option for the loss function to be used.

In [25]:
"""
Quick description of command line parameters used
  --loss_function logistic: sets the model loss function for logistic regression
  --oaa <N>: trains N separate models using One-Against-All approach (all models are captured in the single model file)
             This expects the labels to be contiguous integers starting at 1
  --link logistic: converts the predicted output from logit to probability
The predicted output is the model (label) with the largest likelihood
"""
train_params = 'vw --loss_function logistic --oaa 5 -f {model} -d {data}'.format(model=model_path, data=train_path)
test_params = 'vw --link logistic -i {model} -d {data} -t -p {pred}'.format(model=model_path, data=test_path, pred=prediction_path)

result = run_vw(train_params=train_params,
                test_params=test_params,
                test_data=test,
                prediction_path=prediction_path)

comparison = comparison.append(pd.DataFrame(result, index=['Multinomial Regression']))
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.960662,0.686874,0.261995,0.262026,7.232512,4.612065
Linear Regression w/ Interaction,0.959608,0.688242,0.263613,0.263833,7.564942,4.993011
Multinomial Regression,1.065052,0.718019,0.092890,0.118508,7.649065,4.403493


<h4>Logistic Regression</h4>

Additionally, one might simply be interested in whether the user likes or dislikes an item and we can adjust the input data to represent a binary outcome, where ratings in (1,3] are dislikes (negative results) and (3,5] are likes (positive results).

This framing allows for a simple logistic regression model to be applied. To perform logistic regression the loss_function parameter is changed to 'logistic' and the target label is switched to [0, 1]. Also, be sure to set '--link logistic' during prediction to convert the logit output back to a probability value.

In [26]:
train_params = 'vw --loss_function logistic -f {model} -d {data}'.format(model=model_path, data=train_logistic_path)
test_params = 'vw --link logistic -i {model} -d {data} -t -p {pred}'.format(model=model_path, data=test_logistic_path, pred=prediction_path)

result = run_vw(train_params=train_params,
                test_params=test_params,
                test_data=test,
                prediction_path=prediction_path,
                logistic=True)

comparison = comparison.append(pd.DataFrame(result, index=['Logistic Regression']))
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.960662,0.686874,0.261995,0.262026,7.232512,4.612065
Linear Regression w/ Interaction,0.959608,0.688242,0.263613,0.263833,7.564942,4.993011
Multinomial Regression,1.065052,0.718019,0.092890,0.118508,7.649065,4.403493
Logistic Regression,0.691872,0.382294,0.134096,0.178364,8.637602,4.577976


<h3>Matrix Factorization Based Recommendations</h3>

All of the above approaches train a regression model, but VW also supports matrix factorization with two different approaches.

As opposed to learning direct weights for specific users, items and interactions when training a regression model, matrix factorization attempts to learn latent factors that determine how a user rates an item. An example of how this might work is if you could represent user preference and item categorization by genre. Given a smaller set of genres we can associate how much each item belongs to each genre class, and we can set weights for a user's preference for each genre. Both sets of weights could be represented as a vectors where the inner product would be the user-item rating. Matrix factorization approaches learn low rank matrices for latent features of users and items such that those matrices can be combined to approximate the original user item matrix.

<h4>Singular Value Decomposition Based Matrix Factorization</h4>

The first approach performs matrix factorization based on Singular Value Decomposition (SVD) to learn a low rank approximation for the user-item rating matix. It is is called using the '--rank' command line argument.

See the [Matrix Factorization Example](https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Matrix-factorization-example) for more detail.

In [27]:
"""
Quick description of command line parameters used
  --rank <N>: sets the number of latent factors in the reduced matrix
"""
train_params = 'vw --rank 5 -q ui -f {model} -d {data}'.format(model=model_path, data=train_path)
test_params = 'vw -i {model} -d {data} -t -p {pred}'.format(model=model_path, data=test_path, pred=prediction_path)

result = run_vw(train_params=train_params,
                test_params=test_params,
                test_data=test,
                prediction_path=prediction_path)

comparison = comparison.append(pd.DataFrame(result, index=['Matrix Factorization (Rank)']))
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.960662,0.686874,0.261995,0.262026,7.232512,4.612065
Linear Regression w/ Interaction,0.959608,0.688242,0.263613,0.263833,7.564942,4.993011
Multinomial Regression,1.065052,0.718019,0.092890,0.118508,7.649065,4.403493
Logistic Regression,0.691872,0.382294,0.134096,0.178364,8.637602,4.577976
Matrix Factorization (Rank),1.019621,0.769993,0.168628,0.227570,7.583877,6.538903


<h4>Factorization Machine Based Matrix Factorization</h4>

An alternative approach based on [Rendel's factorization machines](https://cseweb.ucsd.edu/classes/fa17/cse291-b/reading/Rendle2010FM.pdf) is called using '--lrq' (low rank quadratic). More LRQ details in this [demo](https://github.com/VowpalWabbit/vowpal_wabbit/tree/master/demo/movielens).

This learns two lower rank matrices which are multiplied to generate an approximation of the user-item rating matrix. Compressing the matrix in this way leads to learning generalizable factors which avoids some of the limitations of using regression models with extremely sparse interaction features. This can lead to better convergence and smaller on-disk models.

An additional term to improve performance is --lrqdropout which will dropout columns during training. This however tends to increase the optimal rank size. Other parameters such as L2 regularization can help avoid overfitting.

In [28]:
"""
Quick description of command line parameters used
  --lrq <abN>: learns approximations of rank N for the quadratic interaction between namespaces starting with 'a' and 'b'
  --lrqdroupout: performs dropout during training to improve generalization
"""
train_params = 'vw --lrq ui7 -f {model} -d {data}'.format(model=model_path, data=train_path)
test_params = 'vw -i {model} -d {data} -t -p {pred}'.format(model=model_path, data=test_path, pred=prediction_path)

result = run_vw(train_params=train_params,
                test_params=test_params,
                test_data=test,
                prediction_path=prediction_path)

comparison = comparison.append(pd.DataFrame(result, index=['Matrix Factorization (LRQ)']))
comparison

,RMSE,MAE,R2,Explained Variance,Train Time (ms),Test Time (ms)
Linear Regression,0.960662,0.686874,0.261995,0.262026,7.232512,4.612065
Linear Regression w/ Interaction,0.959608,0.688242,0.263613,0.263833,7.564942,4.993011
Multinomial Regression,1.065052,0.718019,0.092890,0.118508,7.649065,4.403493
Logistic Regression,0.691872,0.382294,0.134096,0.178364,8.637602,4.577976
Matrix Factorization (Rank),1.019621,0.769993,0.168628,0.227570,7.583877,6.538903
Matrix Factorization (LRQ),0.979526,0.699106,0.232727,0.232763,6.243971,4.780883


<h4>Conclusion</h4>

The table above shows a few of the approaches in the VW library that can be used for recommendation prediction. The relative performance can change when applied to different datasets and properly tuned, but it is useful to note the rapid speed at which all approaches are able to train (75,000 examples) and test (25,000 examples).

<h3>Scoring</h3>

After training a model with any of the above approaches, the model can be used to score potential user-pairs in offline batch mode, or in a real-time scoring mode. The example below shows how to leverage the utilities in the reco_utils directory to  generate Top-K recommendations from offline scored output.

In [29]:
# store all data
data_path = os.path.join(tmpdir.name, 'all.dat')
to_vw(df=df, output=data_path)

# predict on the full set of users
train_params = 'vw --lrq ui7 --lrqdropout -f {model} -d {data}'.format(model=model_path, data=train_path)
test_params = 'vw -i {model} -d {data} -t -p {pred}'.format(model=model_path, data=data_path, pred=prediction_path)

result = run_vw(train_params=train_params,
                test_params=test_params,
                test_data=df,
                prediction_path=prediction_path)

In [30]:
# load predictions and filter to test set
test_users = [1, 2, 3]
pred_data = pd.read_csv(prediction_path, delim_whitespace=True, names=['prediction'], index_col=1).join(df)
test_user_data = pred_data[pred_data['userID'].isin(test_users)]

top_k = get_top_k_items(test_user_data, col_rating='prediction', k=5)
top_k

,level_0,level_1,prediction,userID,itemID,rating,timestamp
0,0,2,5.0,1,914,3.0,978301968
1,0,8,5.0,1,594,4.0,978302268
2,0,13,5.0,1,2918,4.0,978302124
3,0,21,5.0,1,720,3.0,978300760
4,0,23,5.0,1,527,5.0,978824195
5,1,54,5.0,2,3068,4.0,978299000
6,1,55,5.0,2,1537,4.0,978299620
7,1,57,5.0,2,2194,4.0,978299297
8,1,59,5.0,2,2268,5.0,978299297
9,1,63,5.0,2,3468,5.0,978298542


<h3>Cleanup</h3>

In [31]:
# record results for testing
if is_jupyter():
    for name, value in comparison.loc['Linear Regression w/ Interaction'].to_dict().items():
        pm.record(name.lower(), value)

In [32]:
tmpdir.cleanup()